In [ ]:
%%HTML
<link rel="stylesheet" href="custom.css">

## <span style="color:blue"> Before you start, if you run on colab </span>...
On colab, each notebook runs on its own environment, so you first need to run the following cell in order to upload the required notebooks.

**Advice:** If done yet, don't forget to change the Runtime type into "GPU" so to speed up the computations.


In [ ]:
# First run configuration for colab
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    import os
    print('Colab detected')
    if os.path.isdir('2022-mooc-ecmwf-bocquet-brajard'):
      %cd 2022-mooc-ecmwf-bocquet-brajard/
    if not os.path.isfile('mooc_ecmwf_bocquet_brajard_part1.ipynb'):
      !git clone https://github.com/marcbocquet/2022-mooc-ecmwf-bocquet-brajard.git
      %cd 2022-mooc-ecmwf-bocquet-brajard/

# <span style="color:maroon"> Data assimilation, machine learning and dynamical systems - Part I  </span>
#### Marc Bocquet¹ [marc.bocquet@enpc.fr](mailto:marc.bocquet@enpc.fr) and Julien Brajard² [julien.brajard@nersc.no](mailto:julien.brajard@nersc.no)
#### (1) CEREA, École des Ponts et EdF R&D, Île-de-France, France
#### (2) Nansen Center (NERSC), Bergen, Norway

During this session, we will discover some connections between data assimilation and deep learning when applied to dynamical systems.

# <span style="color:teal"> First step: consistent model trajectory through data assimilation </span>

## <span style="color:blue"> I. The Lorenz 1996 model </span>

The Lorenz 1996 (L96, [Lorenz and Emanuel 1998]) is a low-order chaotic model commonly used in data assimilation to asse the performance of new algorithms. It represents the evolution of some dynamical meteorological scalar variable over a latitude circular band.

The model **dynamics** is driven by the following set of ordinary differential equations:
$$
    \forall n \in [1, N_{\mathsf{x}}], \quad \frac{\mathrm{d}x_{n}}{\mathrm{d}t} =
    (x_{n+1}-x_{n-2})x_{n-1}-x_{n}+F,
$$
where the indices are periodic: $x_{-1}=x_{N_{\mathsf{x}}-1}$, $x_{0}=x_{N_{\mathsf{x}}}$, and $x_{1}=x_{N_{\mathsf{x}}+1}$, and where the system size $N_{\mathsf{x}}$ can take arbitrary values.

In the standard configuration, $N_{\mathsf{x}}=40$ and the forcing coefficient is $F=8$. The ODEs are integrated using a fourth-order Runge-Kutta scheme with a time step of $0.05$ model time unit (MTU). The resulting dynamics are **chaotic** with a doubling time of errors around $0.42$ MTU. For comparison, $0.05$ MTU represents six hours of real time and corresponds to an average autocorrelation of about $0.967$.

## <span style="color:blue"> II. Physical model integration </span>

We now make an integration of the model to visualise a state trajectory (Hovmöller plot) of the L96 model.
We first import standard modules, graphical package and the model which is a numerical implementation of the above ODEs.

In [ ]:
# Import standard modules
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm.notebook import trange, tqdm
import pathlib

# Set random seed
np.random.seed(314)

# Folder for results
folder = dict(output = pathlib.Path("./Output"))
folder['output'].mkdir(exist_ok=True)

# For plot customisation
sns.set_context('notebook')
sns.set_style('darkgrid')
plt.rc('axes', linewidth=1)
plt.rc('axes', edgecolor='k')
plt.rc('figure', dpi=100)
palette = sns.color_palette('deep')

# Import custom L96 model package
from Model.lorenz96 import l96

In the following cell, we define the parameter values of L96 which are taken from the original Lorenz and Emmanuel' paper:
- the number of variables $N_{\mathsf{x}}$ is set to $N_{\mathsf{x}} =40$;
- the forcing coefficient $F$ is set to $F=8$;
- the integration time step is set to $dt=0.05$.

In this configuration, the model variability (spatial average of the time standard deviation per variable) is $3.64$, the doubling time is $0.42$ MTU and the Lyapunov time is $0.61$ MTU. These values might be used later to normalise our plots.

In [ ]:
# Set the parameters of L96 and build the model
Nx = 40
dt = 0.05
F = 8
phi = l96(Nx, dt, F)
phi_stat = dict( model_mean = 2.22,
                 model_std = 3.73,
                 doubling_time = 0.42,
                 lyap_time = 0.61
               )

In the following cell, we perform a rather short model integration, in order to illustrate the model dynamics. The initial condition is a random field.

In [ ]:
# Perform a short model integration (for plotting)
Nt_plot = 500
xt_plot = np.zeros((Nt_plot, Nx))
x = 3 + np.random.randn(Nx)
for t in trange(Nt_plot, desc='physical model integration'):
    x += phi(x)
    xt_plot[t] = x
xt_init = x.copy()

In [ ]:
# Plot model integration
fig = plt.figure(figsize=(18, 6))
plt.grid(False)
im = plt.imshow(xt_plot.T, 
           aspect = 'auto',
           origin = 'lower',
           interpolation = 'spline36',
           cmap = sns.diverging_palette(240, 60, as_cmap=True),
           extent = [0, dt*xt_plot.shape[0], 0, Nx],
           vmin = -10,
           vmax = 15)
plt.colorbar(im)
plt.xlabel('Time (MTU)')
plt.ylabel('Lorenz 96 variables')
plt.tick_params(direction='out', left=True, bottom=True)
plt.show()

## <span style="color:blue"> III. True model definition </span>

Here, we consider the L96 model, which we call here the **physical model**,
as an approximation of a more complex model, the **true model**, which can only be sparsely and noisily observed. We have devised a non-trivial model for this MOOC whose approximation could be the physcal model, i.e. L96. We wanted to avoid employing a true model where only a few parameters of L96 are changed, since this can be very easily learned by traditional data assimilation.

These true model dynamics are driven by the following set of ordinary differential equations (ODEs):
$$
    \forall n \in [1, N_{\mathsf{x}}], \quad \frac{\mathrm{d}x_{n}}{\mathrm{d}t} =
    (x_{n+1}-x_{n-2})x_{n-1} + \nu \left\{ x_{n+1}x_{n-1}(x_{n+2}-x_{n-2}) - x_{n+2}x_n(x_{n+3}-x_{n-1})\right\}
    - \lambda x_{n} +  F,
$$
where the indices are periodic: $x_{-1}=x_{N_{\mathsf{x}}-1}$, $x_{0}=x_{N_{\mathsf{x}}}$, and $x_{1}=x_{N_{\mathsf{x}}+1}$, and where the system size $N_{\mathsf{x}}=40$.

Moreover:
- the forcing coefficient is chosen to be $F=8.5$ as opposed to $8$ in L96;
- the friction coefficient is $\lambda=1.01$ as opposed to $1$ in L96;
- more interestingly, there is an additional cubic term compared to L96 in the tendencies, controlled by the $\nu$ parameter. This term (i) is second-order as in the sense that it is asymptotically consistent with a term of the form $x (\partial x)^2$ (ii) and it conserves the energy $\sum_{n=1}^{N_\mathsf{x}} x_n^2$. Depending on the sign of $\nu$, it could soften or stiffen the L96 model. We choose $\nu=0.025$.

Like for the L96 model, these ODEs are integrated using a fourth-order Runge-Kutta scheme with a time step of $0.05$ model time unit (MTU). The resulting dynamics is **chaotic** with a doubling time of errors around $0.57$ MTU.

In [ ]:
# import custom avection model package
from Model.lorenz96_cubic import l96cb

# Define the true model as a cubic variant of the L96 model
Nx = 40
dt = 0.05
F = 8.5
diffusion = 0.025
friction = 1.01
tmodel = l96cb(Nx, dt, F, diffusion, friction)

# perform a short model integration (for plotting)
Nt_plot = 500
xt_plot = np.zeros((Nt_plot, Nx))
x = xt_init.copy()
for t in trange(Nt_plot, desc='true model integration'):
    x += tmodel(x)
    xt_plot[t] = x

In [ ]:
# plot model integration
fig = plt.figure(figsize=(18, 6))
plt.grid(False)
im = plt.imshow(xt_plot.T, 
           aspect = 'auto',
           origin = 'lower',
           interpolation = 'spline36',
           cmap = sns.diverging_palette(240, 60, as_cmap=True),
           extent = [0, dt*xt_plot.shape[0], 0, Nx],
           vmin = -10,
           vmax = 15)
plt.colorbar(im)
plt.xlabel('Time (MTU)')
plt.ylabel('Lorenz 96 variables')
plt.tick_params(direction='out', left=True, bottom=True)
plt.show()

For those of you interested in the long-term dynamical properties of this model compared to the L96, we can compute and plot the Lyapunov spectra of both models on the same figure for comparison. We observe from both their Hovmöller plot and from their Lyapunov spectrum that indeed the true model with the chosen parameter values is a less unstable and smoother than the original L96 model (our physical model).

In [ ]:
from LyapunovSpectrum.lyaspec import load
index, l96_lyaspec = load('l96_lyaspec.dat')
index, l96cb_lyaspec = load('l96cb_lyaspec.dat')
fig = plt.figure(figsize=(8, 5))
plt.axhline(y=0, color='m', linestyle='dashed')
plt.plot(index, l96_lyaspec, marker="o", label='Lorens96')
plt.plot(index, l96cb_lyaspec, marker="s", label='Lorenz96_cubic')
plt.legend()
plt.xlabel('Lyapunov exponent index')
plt.ylabel('Lyapunov exponent value')
plt.tick_params(direction='out', left=True, bottom=True)
plt.show()

Some statistics of this model can be computed, for comparison to the L96 model and if normalisation is required later on:

In [ ]:
truth_stat = dict( model_mean = 2.12,
                   model_std = 3.54,
                   doubling_time = 0.57,
                   lyap_time = 0.82
                  )

## <span style="color:blue"> IV. Database of observations </span>

Let us first compute a long trajectory of the truth which will be key to our synthetic experiment.
It is locally saved on disk but it will only be accessible through observation for model error discovery.

In [ ]:
# Compute the truth trajectory
Nt = 5000
xt = np.zeros((Nt, Nx), dtype=np.float32)
dates = np.array(range(Nt), dtype=int)
x = xt_init.copy()
for t in trange(Nt, desc='true model integration'):
    x += tmodel(x)
    xt[t] = x
with open(folder['output']/'xt.npy', 'wb') as file:
    np.save(file, xt)
with open(folder['output']/'dates.npy', 'wb') as file:
    np.save(file, dates)

In the data assimilation run, only observations of this trajectory will be generated.
To match more realistic conditions, we only observe one every two grid points, and these scalar observations are noisy. Moreover, from one observation time to the next, these grid points are shifted by one grid cell.
Hence, the observation mask in space and time follows a checkerboard pattern. 
For the sake of simplicity, $N_\mathsf{x}$ is assumed even in the following so that one have the same number of observation at each date, i.e. $N_\mathsf{y}=N_\mathsf{x}/2$.

We add a normal i.i.d observation perturbation to each scalar observation, so that
$\mathbf{R}_k = \sigma_\mathrm{obs}^2 \mathbf{I}_{N_\mathsf{x}}$, with $\sigma_\mathrm{obs}=0.2$.

Let us now compute and save on disk this observational database extracted from the trajectory of the true model.

In [ ]:
# Build the observation operator
from DataAssimilation.observation import checkerboard
H = checkerboard(Nx)

# Compute the observation database
Ny = H.Ny[0]
sig_obs = 0.2
yp = np.empty((Nt, Ny), dtype=np.float32)
for t in trange(Nt, desc='generating observations'):
    yp[t] = H(xt[t], t) + np.random.normal(0, sig_obs, Ny)
with open(folder['output']/'yp.npy', 'wb') as file:
    np.save(file, yp)

At this point, we have at our disposal, a noisy and sparse observational database of the truth $\mathbf{y}_\mathsf{p}$ on the one hand, and the physical model $\Phi$, an approximation of the true model, on the other hand. The goal is to learn a correction to $\Phi$ to get closer to the dynamics of the true model. To do so, we want to apply the very first step of our combined DA+ML algorithm.

The first step is to apply data assimilation with the goal to reconstruct a full field of the system.
The second step, in part II of this MOOC, will apply machine learning to this reconstructed full field.

## <span style="color:blue"> IV. Data assimilation with an ensemble Kalman filter </span>

The data assimilation method that we choose is a deterministic EnKF, the ensemble transform Kalman filter (ETKF, [Hunt et al 2007]) to the problem, with an ensemble choosen to be $N_\mathrm{e}=40$ to avoid the need for localisation, only for the sake of simplicity (see [Malartic et al. 2022] for generalisations with localisation).

A gentle introduction to the ETKF at a master level can be found in chapter 5 of http://cerea.enpc.fr/HomePages/bocquet/teaching/assim-mb-en.pdf.

We assume that we know the observation error statistics, in particular $\sigma_\mathrm{obs}=0.2$.

In addition, we incorporate in the scheme a way to account for model error, i.e. the fact that $\Phi$ is an approximation of the true dynamics. In the absence of such scheme, the EnKF would diverge, being overconfident in its estimation of the errors.

This is achieved by
- the SQRT-CORE scheme [Raanes et al, 2015] with the simplest i.i.d. assumption: $\mathbf{Q}_k = \sigma_q^2 \mathrm{I}$; We choose to use $\sigma_q^2=0.3 \, dt$.
- and via a state-of-the-art adaptive inflation scheme [Raanes et al, 2019].

This just tells the EnKF to be cautious about its estimation of uncertainty and apply a margin of error to be on the safer side, and hence avoid divergence of the EnKF.


In [ ]:
from DataAssimilation.enkf_call import call
Ne = 40                  # Ensemble size
sig_q = np.sqrt(0.3*dt)  # Magnitude for the SQR-CORE correction
Nt = yp.shape[0]         # Length of the DA run  
Nts = 1000               # Spinup 
sig_x = 0.2              # spread of the intial ensemble and std of its mean.
infl = 1                 # mulitplicative inflation

models = (phi, xt, H, yp, dates)
params = (Nx, dt, Nt, Nts, Ne, sig_obs, sig_x, sig_q, infl)

print("> run")
xa, rmse_mean, spread_mean, rmsei_mean, zeta_mean, wctime, prtime = call(models, params)
print("> wctime(s)", wctime, " prtime(s)", prtime)
print("> rmse_mean", rmse_mean, "spread_mean", spread_mean, "rmsei_mean", rmsei_mean)
print("> zeta_mean", zeta_mean)

# Save the analysis trajectory
with open(folder['output']/'xa.npy', 'wb') as file:
    np.save(file, xa)

Because this is a twin experiment, it is possible to check how close to the true full trajectory is the analysed trajectory. The root mean square error (RMSE) between the analysis and the truth should be about $0.20$, or the same order as the observation error of $0.2$ but as an average over both the gauged and ungauged grid points, and with an approximate physical model.

Within the data assimilation run, there was also an estimation of the error made by a cubic interpolation over the ungauged grid points. The RMSE for all sites as compared to the truth should be about $2.71$, and slightly below $1$ for the observed grid points, as expected. 

$\overline{\zeta} \approx 29$ is a measure of the effective size of the ensemble as diagnosed by our adaptive hybrid EnKF-N scheme.

## <span style="color:blue"> V. Provisional conclusion, on our way to part II  </span>

Hence, it is clear that the data assimilation run is worth using. Note that we have not used any smoother data assimilation method, which could have be more complex but also more accurate.

We can now convey the analysis trajectory to the machine learning step, which is part II of our MOOC.
All the necessary fields are stored in "./Ouput".